## Install Glove

In [1]:
!git clone git@github.com:stanfordnlp/GloVe.git
%cd gloVe
!make
%cd ..

Cloning into 'GloVe'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 493 (delta 0), reused 2 (delta 0), pack-reused 486
Receiving objects: 100% (493/493), 199.00 KiB | 209.00 KiB/s, done.
Resolving deltas: 100% (272/272), done.
/Users/davidlassner/research/conferences_and_papers/workshops/2020 DHd/website/material/notebooks/GloVe
mkdir -p build
gcc -c src/vocab_count.c -o build/vocab_count.o -lm -pthread -O3 -march=native -funroll-loops -Wall -Wextra -Wpedantic
clang: warning: -lm: 'linker' input unused [-Wunused-command-line-argument]
gcc -c src/cooccur.c -o build/cooccur.o -lm -pthread -O3 -march=native -funroll-loops -Wall -Wextra -Wpedantic
clang: warning: -lm: 'linker' input unused [-Wunused-command-line-argument]
gcc -c src/shuffle.c -o build/shuffle.o -lm -pthread -O3 -march=native -funroll-loops -Wall -Wextra -Wpedantic
clang: warning: -lm: 'linker' input unused [-Wunused-command-l

## Download the aggressive wikipedia comments data set

In [2]:
!mkdir data
!wget https://ndownloader.figshare.com/files/7394506 -O agression_annotation.tsv
!wget https://ndownloader.figshare.com/files/7038038 -O aggression_annotated_comments.tsv
!mv agression_annotation.tsv data/
!mv aggression_annotated_comments.tsv data/

--2020-01-31 11:43:41--  https://ndownloader.figshare.com/files/7394506
Resolving ndownloader.figshare.com (ndownloader.figshare.com)... 34.253.86.128, 54.171.25.236, 108.128.77.47, ...
Connecting to ndownloader.figshare.com (ndownloader.figshare.com)|34.253.86.128|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/7394506/aggression_annotations.tsv [following]
--2020-01-31 11:43:41--  https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/7394506/aggression_annotations.tsv
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.36.218
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.36.218|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30627328 (29M) [binary/octet-stream]
Saving to: ‘agression_annotation.tsv’

agression_annotatio 100%[===================>]  29.21M  42.8MB/s    in 0.7s    

2020-01-31 11:43:42 (42.8 MB/s) - ‘agression

## Preprocess data

In [3]:
import pandas as pd
from tqdm.notebook import tqdm
aggression_annotated_comments = pd.read_csv("data/aggression_annotated_comments.tsv", sep="\t")
agression_annotation = pd.read_csv("data/agression_annotation.tsv", sep="\t")

agression_data = pd.merge(aggression_annotated_comments, agression_annotation, on="rev_id")

In [4]:
comments = []
labels = []
for rev_id, rev in tqdm(agression_data.groupby("rev_id")):
    comments.append(rev.iloc[0].comment)
    labels.append(rev.aggression.sum()/len(rev) >.5)

In [5]:
from sklearn.model_selection import train_test_split    
train_embeddings, X_other, embeddings_labels, y_other  = train_test_split(comments, labels)
X_train, X_test, y_train, y_test = train_test_split(X_other, y_other)

In [6]:
from spacy.lang.en import English
nlp = English()
tokenizer = nlp.Defaults.create_tokenizer(nlp)

with open("data/all_comments_plain.txt", "w") as all_fout:
    with open("data/aggressive_comments_plain.txt", "w") as agg_fout:
        with open("data/non_aggressive_comments_plain.txt", "w") as non_agg_fout:
            for comment, label in tqdm(
                zip(tokenizer.pipe(train_embeddings), embeddings_labels),
                total=len(train_embeddings),
                desc="storing comments"
            ):
                out_line = " ".join([t.text.replace("\n", "\t") for t in comment])
                all_fout.write("{}\n".format(out_line))
                if label == 1:
                    agg_fout.write("{}\n".format(out_line))
                else:
                    non_agg_fout.write("{}\n".format(out_line))


In [7]:
!ls data

aggression_annotated_comments.tsv all_comments_plain.txt
aggressive_comments_plain.txt     non_aggressive_comments_plain.txt
agression_annotation.tsv


In [8]:
!ls gloVe/build/

common.o      cooccur.o     glove.o       shuffle.o     vocab_count.o
cooccur       glove         shuffle       vocab_count


## Build Vocabulary

In [9]:
!gloVe/build/vocab_count -max-vocab 25000 -min-count 10 < data/all_comments_plain.txt > data/vocab.txt 

BUILDING VOCABULARY
Processed 0 tokens.100000 tokens.200000 tokens.300000 tokens.400000 tokens.500000 tokens.600000 tokens.700000 tokens.800000 tokens.900000 tokens.1000000 tokens.1100000 tokens.1200000 tokens.1300000 tokens.1400000 tokens.1500000 tokens.1600000 tokens.1700000 tokens.1800000 tokens.1900000 tokens.2000000 tokens.2100000 tokens.2200000 tokens.2300000 tokens.2400000 tokens.2500000 tokens.2600000 tokens.2700000 tokens.2800000 tokens.2900000 tokens.3000000 tokens.3100000 tokens.3200000 tokens.3300000 tokens.3400000 tokens.3500000 tokens.3600000 tokens.3700000 tokens.3800000 tokens.3900000 tokens.4000000 tokens.4100000 tokens.4200000 tokens.4300000 tokens.4400000 tokens.4500000 tokens.4600000 tokens.4700000 tokens.4800000 tokens.4900000 tokens.5000000 tokens.5100000 tokens.5200000 tokens.5300000 tokens.5400000 tokens.5500000 tokens.5600000 tokens.5700000 tokens.5800000 tokens.5900000 tokens.6000000 tokens.6100000 tokens.6200000 tokens.6300000 tokens.6400000 tokens.6500000 to

## Create cooccurrences

In [10]:
!gloVe/build/cooccur -vocab-file data/vocab.txt < data/non_aggressive_comments_plain.txt > data/non_aggressive_comments_cooccurrences.bin
!gloVe/build/cooccur -vocab-file data/vocab.txt < data/aggressive_comments_plain.txt > data/aggressive_comments_cooccurrences.bin
!gloVe/build/cooccur -vocab-file data/vocab.txt < data/all_comments_plain.txt > data/all_comments_cooccurrences.bin

COUNTING COOCCURRENCES
window size: 15
context: symmetric
max product: 10485784
overflow length: 28521267
Reading vocab from file "data/vocab.txt"...loaded 23261 words.
Building lookup table...table contains 51813977 elements.
Processing token: 010000020000030000040000050000060000070000080000090000010000001100000120000013000001400000150000016000001700000180000019000002000000210000022000002300000240000025000002600000270000028000002900000300000031000003200000330000034000003500000360000037000003800000390000040000004100000420000043000004400000450000046000004700000480000049000005000000510000052000005300000540000055000005600000570000058000005900000600000061000006200000630000064000006500000Processed 6510651 tokens.
Writing cooccurrences to disk........2 files in total.
Merging cooccurrence files: processed 0 lines.100000 lines.200000 lines.300000 lines.400000 lines.500000 lines.600000 lines.700000 lines.800000 lines.900000 lines.1000000 lines.1100000 lines.1200000 lines.1300000 lines.1400000 

Merging cooccurrence files: processed 0 lines.100000 lines.200000 lines.300000 lines.400000 lines.500000 lines.600000 lines.700000 lines.800000 lines.900000 lines.1000000 lines.1100000 lines.1200000 lines.1300000 lines.1400000 lines.1500000 lines.1600000 lines.1700000 lines.1800000 lines.1900000 lines.2000000 lines.2100000 lines.2200000 lines.2300000 lines.2400000 lines.2500000 lines.2600000 lines.2700000 lines.2800000 lines.2900000 lines.3000000 lines.3100000 lines.3200000 lines.3300000 lines.3400000 lines.3500000 lines.3600000 lines.3700000 lines.3800000 lines.3900000 lines.4000000 lines.4100000 lines.4200000 lines.4300000 lines.4400000 lines.4500000 lines.4600000 lines.4700000 lines.4800000 lines.4900000 lines.5000000 lines.5100000 lines.5200000 lines.5300000 lines.5400000 lines.5500000 lines.5600000 lines.5700000 lines.5800000 lines.5900000 lines.6000000 lines.6100000 lines.6200000 lines.6300000 lines.6400000 lines.6500000 lines.6600000 lines.6700000 lines.6800000 lines.6900000 lin

In [11]:
!gloVe/build/shuffle -verbose 2 -memory 4 < data/non_aggressive_comments_cooccurrences.bin > data/non_aggressive_comments_cooccurrences_shuffled.bin
!gloVe/build/shuffle -verbose 2 -memory 4 < data/aggressive_comments_cooccurrences.bin > data/aggressive_comments_cooccurrences_shuffled.bin
!gloVe/build/shuffle -verbose 2 -memory 4 < data/all_comments_cooccurrences.bin > data/all_comments_cooccurrences_shuffled.bin

Using random seed 1580467654
SHUFFLING COOCCURRENCES
array size: 255013683
Shuffling by chunks: processed 0 lines.processed 15858731 lines.
Wrote 1 temporary file(s).
Merging temp files: processed 0 lines.15858731 lines.Merging temp files: processed 15858731 lines.

Using random seed 1580467661
SHUFFLING COOCCURRENCES
array size: 255013683
Shuffling by chunks: processed 0 lines.processed 2927335 lines.
Wrote 1 temporary file(s).
Merging temp files: processed 0 lines.2927335 lines.Merging temp files: processed 2927335 lines.

Using random seed 1580467662
SHUFFLING COOCCURRENCES
array size: 255013683
Shuffling by chunks: processed 0 lines.processed 16816576 lines.
Wrote 1 temporary file(s).
Merging temp files: processed 0 lines.16816576 lines.Merging temp files: processed 16816576 lines.



## Train Glove

In [12]:
!gloVe/build/glove -input-file data/non_aggressive_comments_cooccurrences_shuffled.bin -vocab-file data/vocab.txt -save-file data/non_aggressive_comments_vec -verbose 2 -vector-size 32 -threads 8 -alpha 0.75 -x-max 100.0 -eta 0.05 -binary 0 -model 2
!gloVe/build/glove -input-file data/aggressive_comments_cooccurrences_shuffled.bin -vocab-file data/vocab.txt -save-file data/aggressive_comments_vec -verbose 2 -vector-size 32 -threads 8 -alpha 0.75 -x-max 100.0 -eta 0.05 -binary 0 -model 2
!gloVe/build/glove -input-file data/all_comments_cooccurrences_shuffled.bin -vocab-file data/vocab.txt -save-file data/all_comments_vec -verbose 2 -vector-size 32 -threads 8 -alpha 0.75 -x-max 100.0 -eta 0.05 -binary 0 -model 2

TRAINING MODEL
Read 15858731 lines.
Initializing parameters...Using random seed 1580467674
done.
vector size: 32
vocab size: 23261
x_max: 100.000000
alpha: 0.750000
01/31/20 - 11:47.56AM, iter: 001, cost: 0.021714
01/31/20 - 11:47.57AM, iter: 002, cost: 0.014477
01/31/20 - 11:47.58AM, iter: 003, cost: 0.012972
01/31/20 - 11:48.00AM, iter: 004, cost: 0.011847
01/31/20 - 11:48.01AM, iter: 005, cost: 0.010900
01/31/20 - 11:48.03AM, iter: 006, cost: 0.010162
01/31/20 - 11:48.05AM, iter: 007, cost: 0.009618
01/31/20 - 11:48.06AM, iter: 008, cost: 0.009218
01/31/20 - 11:48.08AM, iter: 009, cost: 0.008911
01/31/20 - 11:48.09AM, iter: 010, cost: 0.008664
01/31/20 - 11:48.11AM, iter: 011, cost: 0.008464
01/31/20 - 11:48.12AM, iter: 012, cost: 0.008297
01/31/20 - 11:48.14AM, iter: 013, cost: 0.008155
01/31/20 - 11:48.16AM, iter: 014, cost: 0.008031
01/31/20 - 11:48.18AM, iter: 015, cost: 0.007925
01/31/20 - 11:48.19AM, iter: 016, cost: 0.007832
01/31/20 - 11:48.21AM, iter: 017, cost: 0.007749
01

## Load Embeddings

In [13]:
from bisect import bisect_left
import numpy as np

with open("data/vocab.txt") as fin:
    vocab,_ = zip(*map(lambda x: x.split(" "), fin))
vocab = sorted(list(vocab) + ["<unk>"])

In [14]:
def index(a, x):
    'Locate the leftmost value exactly equal to x'
    i = bisect_left(a, x)
    if i != len(a) and a[i] == x:
        return i
    raise ValueError

def load_embedding(path_, vocab):
    dim = 32
    mat = np.empty((len(vocab), dim))
    
    with open(path_) as fin:
        for row in tqdm(fin):
            splitted = row.replace("\n", "").split(" ")
            key, vec = splitted[0], splitted[1:]
            mat[index(vocab, key)] = vec
            
    return mat
            
all_vec = load_embedding("data/all_comments_vec.txt", vocab)
aggressive_vec = load_embedding("data/aggressive_comments_vec.txt", vocab)
non_aggressive_vec = load_embedding("data/non_aggressive_comments_vec.txt", vocab)

In [29]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

def embed_X(X, vocab, embedding, verbose=True):
    embedded_X = np.zeros((len(X), embedding.shape[1]))
    num_misses = 0
    num_hits = 0
    for icomment, comment in tqdm(enumerate(tokenizer.pipe(X)), total=len(X)):
        for token in comment:
            try:
                embedded_X[icomment] += embedding[index(vocab, token.text.replace("\n", "\t"))]
                num_hits += 1
            except ValueError:
                num_misses += 1
                if verbose:
                    print("{} not in vocab".format(token))
        embedded_X[icomment] /= len(comment)
    
    print("misses/hits {}".format(num_misses/num_hits))
    return embedded_X

def train(X,y):
      
    clf = KNeighborsClassifier()
    
    clf.fit(X, y)
    return clf

def evaluate(clf, X, y_true):
    y_pred = clf.predict(X)
    print(classification_report(y_true, y_pred))

print("all")

clf = train(embed_X(X_train, vocab, all_vec, verbose=False), y_train)
evaluate(clf, embed_X(X_test, vocab, all_vec, verbose=False), y_test)

print("aggressive_vec")

clf = train(embed_X(X_train, vocab, aggressive_vec, verbose=False), y_train)
evaluate(clf, embed_X(X_test, vocab, aggressive_vec, verbose=False), y_test)

print("non_aggressive_vec")

clf = train(embed_X(X_train, vocab, non_aggressive_vec, verbose=False), y_train)
evaluate(clf, embed_X(X_test, vocab, non_aggressive_vec, verbose=False), y_test)

all



misses/hits 0.08105129530252014



misses/hits 0.08537588390176032
              precision    recall  f1-score   support

       False       0.92      0.97      0.94      6327
        True       0.65      0.45      0.53       915

    accuracy                           0.90      7242
   macro avg       0.79      0.71      0.74      7242
weighted avg       0.89      0.90      0.89      7242

aggressive_vec



misses/hits 0.08105129530252014



misses/hits 0.08537588390176032
              precision    recall  f1-score   support

       False       0.91      0.98      0.94      6327
        True       0.69      0.34      0.46       915

    accuracy                           0.90      7242
   macro avg       0.80      0.66      0.70      7242
weighted avg       0.88      0.90      0.88      7242

non_aggressive_vec



misses/hits 0.08105129530252014



misses/hits 0.08537588390176032
              precision    recall  f1-score   support

       False       0.92      0.95      0.93      6327
        True       0.55      0.42      0.47       915

    accuracy                           0.88      7242
   macro avg       0.73      0.68      0.70      7242
weighted avg       0.87      0.88      0.88      7242

